# The following script prepares the file MP.csv from the MasteringPhysics exported file

# Steps:

- Download masteringphysics  as MP_raw.csv. 

- The following code will create the file MP.csv for use in the next script 
- WARNING: Make sure there are no NaN's (...'s )in the columns of MP_raw (this might happen if a student has been given a deadline extension. Replace the (...) by a 0 before running the script

In [1]:
import pandas as pd
import numpy as np

inputfile = 'MP_raw.csv'
outputfile = 'MP.csv'
df=pd.read_csv(inputfile, skiprows=3)#reads the file skipping first 3 rows

df = df.replace(np.nan, 0)#replaces original NaN's by 0 so these columns are not removed in th next steps
df = df.replace('--', np.nan)# mark the --'s by Nan's to nuke them
df = df.dropna(axis='columns')
df = df[:-1]# removes the useless last row

# columns_to_drop = ['Login Name', 'Email', 'Enrollment Status', 'Group(s)']
columns_to_drop = ['Enrollment Status', 'Group(s)']
df = df.drop(columns_to_drop, axis=1)

HW_cols = [x for x in list(df) if 'HW' in x]
df['Total Grade'] = df[HW_cols].sum(axis=1)
total = df["Total Grade"][df.index[-1]]
df['MP Score'] = 100*df['Total Grade']/total
df.to_csv(outputfile, index=False)

# The following script transfers MP grades to the (downloaded) BS Gradebook

# Steps:

- Make sure BSGrades contains the orgDefinedId

- Run the previous script to convert MP_raw.csv to MP.csv which is properly formatted for processing. 

- Make sure Brightspace gradebook is downloaded as BSGrades.csv, and "out of" for tests are correct. 

- Set to_print='y' [CLOSE BSGrades.csv if its open]

- If Brightspace says "invalid value" try hand-typing the first value


In [2]:
import pandas as pd
import os

df=pd.read_csv('BSGrades.csv')
cols=list(df)
assert "OrgDefinedId" in cols, "Make sure there is OrgDefinedId in cols"

# stripping off the # from in front of the org defined id
dfcopy =df.copy()
df["OrgDefinedId"]=df["OrgDefinedId"].apply(lambda x: x[1:] if x[0]=="#" else x)

#This part finds the name of the column that has the scores. Assumes that this column has MP in its title
df_mp=pd.read_csv('MP.csv')
cols_mp = list(df_mp)
cols_named_MP=[x for x in cols_mp if "MP" in x]
assert len(cols_named_MP) == 1, "Make sure the Mastering Scores are in column titled MP Score. No other column should have MP in its header"
MPScores = cols_named_MP[0]

df_mp=df_mp.rename(columns={'Student ID': 'OrgDefinedId'})

df = pd.merge(df,df_mp[['OrgDefinedId',MPScores]],on='OrgDefinedId', how='left')

dfcopy['Mastering Physics Points Grade <Numeric MaxPoints:100 Weight:14.75>']=df[MPScores]

try:
    dfcopy.to_csv('BSGrades.csv', index=False)
    print("Task completed successfully")
except:
    print('Close BSGrades.csv')

Task completed successfully


# The following script calculates letter grades from the numerical grades, and also applies the reward for good dropped midterm. 

* Create the final file as BSGrades_final.csv (this is after uploading the final exam grades and having BS calculate overall grades)
* Adds a column "Adjusted Numerical Grade" which includes the reward for the good dropped midterm. 
* Automatically figures out the available number of points in each midterm.
* Calculates letter grades based on this and stores in a new column "Letter"


In [1]:
import pandas as pd
import re
def lg(num_grade):
    if num_grade<59.5:
        return 'F'
    elif num_grade>=59.5 and num_grade<62.5:
        return 'D-'
    elif num_grade>=62.5 and num_grade<66.5:
        return 'D'
    elif num_grade>=66.5 and num_grade<69.5:
        return 'D+'
    elif num_grade>=69.5 and num_grade<72.5:
        return 'C-'
    elif num_grade>=72.5 and num_grade<76.5:
        return 'C'
    elif num_grade>=76.5 and num_grade<79.5:
        return 'C+'
    elif num_grade>=79.5 and num_grade<82.5:
        return 'B-'
    elif num_grade>=82.5 and num_grade<86.5:
        return 'B'
    elif num_grade>=86.5 and num_grade<89.5:
        return 'B+'
    elif num_grade>=89.5 and num_grade<92.5:
        return 'A-'
    else:
        return 'A'

def f(x, mta):
    pointmax0=mta[0]
    pointmax1=mta[1]
    pointmax2=mta[2]
    min_midterm = min(x[0]/pointmax0,x[1]/pointmax1,x[2]/pointmax2)
    if min_midterm>=0.7 and min_midterm<0.8:
        return x[3]+.5
    elif min_midterm>=0.8:
        return x[3]+1
    else:
        return x[3]

def striphash(x):
    return x[1:]



df=pd.read_csv('BSGrades_final.csv')
columns = ['Calculated Final Grade Denominator','Adjusted Final Grade Numerator','Adjusted Final Grade Denominator','End-of-Line Indicator']
df.drop(columns, inplace=True, axis=1)
cols=list(df)
testcols = [x for x in cols if "Category:Midterms" in x] # These columns have the test scores
#Make a dictionary which will hold midterm number and total points for that midterm
midterm_totals = {}
for i in range(len(testcols)):
    midterm_no = int(re.findall(r'Midterm\s+\d+|Midterm\d+', testcols[i])[0][-1])
    _ = re.findall(r'MaxPoints:\d+', testcols[i])[0]
    midterm_pts = int(re.findall(r'\d+', _)[0])
    midterm_totals.update( {midterm_no : midterm_pts} )
midterm_totals_array = [midterm_totals[1], midterm_totals[2], midterm_totals[3]]
print(f'Midterm totals detected: {midterm_totals[1]}, {midterm_totals[2]}, {midterm_totals[3]}')



for n in range(1,4):
    testname="Midterm "+str(n)+" Points Grade"
    testname1="Midterm"+str(n)+" Points Grade" #Just in case the Midterm appears and Midterm1 rather than Midterm 1
    shortname="Midterm "+str(n)
    u=[cols.index(x) for x in cols if testname in x or testname1 in x]
    if len(u)>0:
        cols[u[0]]=shortname
    else:
        print(testname+"not found")
df.columns=cols        
df.rename(columns={'Calculated Final Grade Numerator': 'Numerical Grade'},inplace=True)
df["Adjusted Numerical Grade"] = ""
df["Letter"] = ""

df['Adjusted Numerical Grade'] = df[['Midterm 1','Midterm 2','Midterm 3','Numerical Grade']].apply(lambda x: f(x, midterm_totals_array),  axis=1)
df['Letter'] = df['Adjusted Numerical Grade'].map(lg)
df['OrgDefinedId']=df['OrgDefinedId'].map(striphash)
df.rename(columns={'OrgDefinedId': 'VUnetID'},inplace=True)
try:
    df.to_csv('OverallGrades.csv', index=False)
    print('Data written to OverallGrades.csv')
except:
    print('Close OverallGrades.csv and try again')
    
    


Midterm totals detected: 42, 42, 42
Data written to OverallGrades.csv


# The following script transfers final letter grades into the file for uploading to "Online Grading".

* Make sure OverallGrades.csv is ready (previous script)

* Download the file from Online Grading, name it something like "fingra_01",  and set the online_grading_filename variable to the name of this file

* Run the script and upload the file finalgrades.csv

In [4]:
import pandas as pd
df_copyfrom=pd.read_csv('OverallGrades.csv')

online_grading_filename = 'fingra_02.csv'
df=pd.read_csv(online_grading_filename)

df = pd.merge(df,df_copyfrom[['VUnetID','Letter']],on='VUnetID', how='left')
df['Assigned Grade']=df['Letter']
df.drop('Letter', inplace=True, axis=1)
df = df.fillna('')
try:
    df.to_csv(online_grading_filename, index=False)
    print("{} written".format(online_grading_filename))
except:
    print('Close open excel file and try again')

fingra_02.csv written


# The following script calculates the discussion score and updates BSGrades.

* Download the discussion results (Introductions -> View Topic Statistics) as DiscussionGrades.csv and Tips.csv

* Enter the filesnames and appropriate keywords in the dict "discussion"

* Download the gradebook as BSGrades.csv

* Make sure BSGrades first column is "OrgDefinedId" (will give an assert error if not)

* The script will calculate the scores for each discussion and write them in the correct columns

In [ ]:
import pandas as pd
import glob


def calculate_discussion_score(x): 
    threads=x[1]
    replies=x[2]
    return min(1,threads)*8+min(2,replies)


# discussions = {"DiscussionGrades.csv": "Introduction", "Tips.csv": "Study Tips"}
discussions = {"DiscussionGrades.csv": "Introduction"}
files = list(discussions.keys())
df=pd.read_csv('BSGrades.csv')
assert 'OrgDefinedId' in list(df), "Make sure BSGrades has OrgDefinedId as a header"
dfcopy = df.copy()
df.rename(columns={'OrgDefinedId': 'Username'},inplace=True)
df['Username']=df['Username'].map(lambda x: x[1:] if x[0]=='#' else x)
for file in files:
    df_disc=pd.read_csv(file)
    list_of_cols=list(df_disc)
    list_to_keep=['[Username]','[Threads]','[Replies]']
    list_to_drop=[x for x in list_of_cols if not x in list_to_keep]
    df_disc.drop(list_to_drop, inplace=True, axis=1)
    df_disc["Score"]=0.0
    df_disc['Score'] = df_disc.apply(calculate_discussion_score, axis=1)
    df_disc.rename(columns={'[Username]': 'Username'},inplace=True)

    cols=list(df)
    n=[cols.index(x) for x in cols if discussions[file] in x][0]#index of the column in BSGrades containing the keyword for that discussion, e.g. "Introductions" 
    copy_to_column=cols[n]

    df = pd.merge(df,df_disc[['Username','Score']],on='Username', how='left')
    dfcopy[copy_to_column]=df['Score']
    print("{} score transfer complete".format(file))
dfcopy.to_csv('BSGrades.csv', index=False)





# The following script calculates midterm deficiencies for each section and saves as CSV files.

* Download grades as BSGrades. Make sure "Section Membership" is checked

* Change section_prefix and section_suffix appropriately. Adjust for the number of sections

* The script will calculate the midterm deficiencies and make a seoparate file for each section for each discussion and write them in the correct columns

In [ ]:
import pandas as pd
import regex as re

section_prefix='SEC_2022F.1.PHYS.1602.'
section_suffix1='02'
section_suffix2='03'

def lg(num_grade):
    if num_grade<59.5:
        return 'F'
    elif num_grade>=59.5 and num_grade<62.5:
        return 'D-'
    elif num_grade>=62.5 and num_grade<66.5:
        return 'D'
    elif num_grade>=66.5 and num_grade<69.5:
        return 'D+'
    elif num_grade>=69.5 and num_grade<72.5:
        return 'C-'
    elif num_grade>=72.5 and num_grade<76.5:
        return 'C'
    elif num_grade>=76.5 and num_grade<79.5:
        return 'C+'
    elif num_grade>=79.5 and num_grade<82.5:
        return 'B-'
    elif num_grade>=82.5 and num_grade<86.5:
        return 'B'
    elif num_grade>=86.5 and num_grade<89.5:
        return 'B+'
    elif num_grade>=89.5 and num_grade<92.5:
        return 'A-'
    else:
        return 'A'



df = pd.read_csv('BSGrades.csv')
df = df[['OrgDefinedId', 'Last Name', 'First Name','Sections','Calculated Final Grade Numerator', 'Calculated Final Grade Denominator']]
df['Grade'] = 100*df['Calculated Final Grade Numerator']/df['Calculated Final Grade Denominator']
df=df.drop(columns=['Calculated Final Grade Numerator', 'Calculated Final Grade Denominator'])
df = df.replace(to_replace=section_prefix, value='', regex=True)
df['Letter'] = df["Grade"].apply(lg)
df02 = df.loc[(df['Sections']==section_suffix1) & (df['Grade']<72.5)].sort_values(by=['Last Name', 'First Name'])
df03 = df.loc[(df['Sections']==section_suffix2) & (df['Grade']<72.5)].sort_values(by=['Last Name', 'First Name'])
df02.to_csv(f'midterm-deficiencies-section-{section_suffix1}.csv', index=False)
df03.to_csv(f'midterm-deficiencies-section-{section_suffix2}.csv', index=False)

# The following allows the VU-net id to be transferred to the Student-Id field in the MP roster

In [ ]:
import difflib

gradebook='BSGrades.csv'
mpfile = 'MP_roster.csv'
df = pd.read_csv(gradebook)
df=pd.read_csv(gradebook, usecols=["Last Name", "First Name", "Username"])
df['Name']=df['Last Name']+', '+df['First Name']

df_mp=pd.read_csv(mpfile, usecols=["Name", "Student ID"], skiprows=12)

def find_similar(x):
    val = difflib.get_close_matches(str(x), df['Name'])
    if isinstance(val, list) and len(val)>0:
        return val[0]
    return ""
df_mp['Student Matched Name'] = df_mp['Name'].apply(find_similar)

# find the non matches
df_mp[df_mp["Student Matched Name"] == ""]

In [ ]:
# make manual fixes e.g
df_mp.iloc[232]['Student Matched Name'] = "YU JIAHAO"

In [ ]:
merged = pd.merge(df_mp, df, left_on='Student Matched Name', right_on="Name", how="left")
cols = ["Name_x", "Name_y", "Username"]
merged = merged[cols]
merged.to_csv("to_check_names.csv")

* Manually inspect the csv file in excel (use duplicate highlighting)
* check if name in middle col (from BSGrades) matches name in left col ("from MP's roster)
* Make sure the names of students in MP's roster (left col) have correct student id's (right column)
* delete the middle column
* import the resulting file
* copy paste this data into MP_roster.csv

# The following script calculates midterm the questions missed on tests and puts them in the BSGrades file.

* Download grades as BSGrades. 

* Make sure the gradescope output file is ready (By clicking Download Responses in Gradescope)

* Change midterm_num. 1-3 for midterms and 4 for final

* The script will put in the exam score and the questions missed

* If everyone is getting a bonus change bonus_points_for_all

In [1]:
import pandas as pd
import re
midterm_num=4
######Change the next line to 0 unless you want to give bonus points to all
bonus_points_for_all=0
#############################################


df = pd.read_csv("BSGrades.csv")
    

grades_col =""
missed_col =""

if midterm_num == 4:
    df_gs=pd.read_csv('Final_student_responses.csv')
    grades_col = 'Final Points Grade <Numeric MaxPoints:57 Weight:25>'
    missed_col = 'Final Questions Missed Text Grade <Text>'
else:
    df_gs=pd.read_csv(f'Midterm_{midterm_num}_student_responses.csv')
    grades_col = f'Midterm {midterm_num} Points Grade <Numeric MaxPoints:45 Weight:33.333333333 Category:Midterms CategoryWeight:60>'
    missed_col = f"Midterm {midterm_num} Questions Missed Text Grade <Text>"
    

df_gs_colnames =list(df_gs.columns)
bsg_colnames = list(df.columns)# for automatically fining the title of the grades col and missed col TODO


    
#automatically calculating the number of questions using by counting the number of columns titled "Question n Score"
regex = r"Question \d+ Score"
number_of_questions = len([x for x in df_gs_colnames if re.match(regex, x)])
    

def make_missed_list(x):
    if pd.isnull(x["Version"]):
        return ""

    ver ="v"+str(x["Version"])+": "
    missed =[]
    res = ""
    for i in range(1,number_of_questions):
        scorecol=f'Question {i} Score'
        if x[scorecol] == 0:
            missed.append(str(i))
    
    if missed:
        res = ver +",".join(missed)
    else:
        res = ver + "All Correct!"
        
    return res

def add_bonus(x):
    newval = 0
    try:
        newval=float(x)+bonus_points_for_all
    except ValueError:
        pass
    return newval


df_gs["Missed"] = df_gs.apply(lambda row: make_missed_list(row), axis=1)
dfcopy=df.copy()
df["OrgDefinedId"]=df["OrgDefinedId"].map(lambda x: x[1:] if x[0]=='#' else x)
merged = pd.merge(df, df_gs, left_on="OrgDefinedId", right_on='Student ID', how="left")
dfcopy[missed_col] = merged["Missed"]
dfcopy[grades_col] = merged["Total Score"].apply(lambda x: add_bonus(x))
dfcopy=dfcopy.replace('--', '')
dfcopy.to_csv('BSGrades.csv', index=False)

# Development Area

In [ ]:
colnames =list(df_gs.columns)

In [ ]:
import re

regex = r"Question \d+ Score"

len([x for x in colnames if re.match(regex, x)])

In [ ]:
"abcde".startswith(f'Midterm {midterm_num}')

In [ ]:
bsg_colnames = list(df.columns)
[x for x in bsg_colnames if x.startswith(f'Midterm {1} Points Grade')][0]

In [ ]:
colnames